In [ ]:
This notebook uses "DSSP" (need to be installed separately) to calculate the 2nd structure of the PDBs.
The output file "pdb_stats.csv" contains the 2nd info of all the PDBs. It only need to be run once for one PDB dataset.  

In [18]:
import pandas as pd
from glob import glob
import os

pdbs = []

for d in ['001','002','003','004']:
    os.system(f'mkdir -p {d}_dssp')
    pdbs.extend(glob(f"{d}/*pdb"))

for pdb in pdbs:
    out_folder = os.path.dirname(pdb) + "_dssp"
    out_base_name = os.path.basename(pdb).split(".")[0]
    out_name = f'./{out_folder}/{out_base_name}.dssp'
    os.system(f'dssp -i {pdb} -o  {out_name}')

In [118]:
dssp_files = []
for d in ['001_dssp', '002_dssp', '003_dssp', '004_dssp']:
    dssp_files.extend(glob(f'./{d}/*dssp'))

dssp_stats = []
for dssp in dssp_files:
    dssp_base_name = os.path.basename(dssp).split('.')[0] 
    with open(dssp,'r') as f:
        line_count = 0
        structure_part = 0
        H_count = 0
        E_count = 0
        for line in f:
            if structure_part == 1:
                if line[16] == 'H':
                    H_count += 1
                elif line[16] == 'E':
                    E_count += 1
                line_count += 1
            if "#" in line:
                structure_part = 1
        H_percent = round(100*H_count/line_count)
        E_percent = round(100*E_count/line_count)
        for i in [1,2,3]:
            pdb_name = f'{dssp_base_name}_col_best_{i:03d}.pdb'
            dssp_stats.append([pdb_name,line_count, H_count, H_percent, E_count, E_percent])
        
dssp_stats_df = pd.DataFrame(dssp_stats, columns=['name', 'n_residues', 'n_helix', 'helix%', 'n_strand', 'strand%'])
dssp_stats_df.to_csv('pdb_stats.csv', index=False)

In [119]:
len(dssp_stats_df)

64845